In [1]:
# Load env variables and create client
import base64
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-3-7-sonnet-latest"

In [2]:
# Helper functions
from anthropic.types import Message


def add_user_message(messages, message):
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    thinking=False,
    thinking_budget=1024,
):
    params = {
        "model": model,
        "max_tokens": 4000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

In [3]:
# Fire risk assessment prompt
prompt = """
Analyze the attached satellite image of a property with these specific steps:

1. Residence identification: Locate the primary residence on the property by looking for:
   - The largest roofed structure 
   - Typical residential features (driveway connection, regular geometry)
   - Distinction from other structures (garages, sheds, pools)
   Describe the residence's location relative to property boundaries and other features.

2. Tree overhang analysis: Examine all trees near the primary residence:
   - Identify any trees whose canopy extends directly over any portion of the roof
   - Estimate the percentage of roof covered by overhanging branches (0-25%, 25-50%, 50-75%, 75-100%)
   - Note particularly dense areas of overhang

3. Fire risk assessment: For any overhanging trees, evaluate:
   - Potential wildfire vulnerability (ember catch points, continuous fuel paths to structure)
   - Proximity to chimneys, vents, or other roof openings if visible
   - Areas where branches create a "bridge" between wildland vegetation and the structure
   
4. Defensible space identification: Assess the property's overall vegetative structure:
   - Identify if trees connect to form a continuous canopy over or near the home
   - Note any obvious fuel ladders (vegetation that can carry fire from ground to tree to roof)

5. Fire risk rating: Based on your analysis, assign a Fire Risk Rating from 1-4:
   - Rating 1 (Low Risk): No tree branches overhanging the roof, good defensible space around the structure
   - Rating 2 (Moderate Risk): Minimal overhang (<25% of roof), some separation between tree canopies
   - Rating 3 (High Risk): Significant overhang (25-50% of roof), connected tree canopies, multiple points of vulnerability
   - Rating 4 (Severe Risk): Extensive overhang (>50% of roof), dense vegetation against structure, numerous ember catch points, limited defensible space

For each item above (1-5), write one sentence summarizing your findings, with your final response being the numeric Fire Risk Rating (1-4) with a brief justification.
"""

In [4]:
with open("images/prop7.png", "rb") as f:
    image_bytes = base64.standard_b64encode(f.read()).decode("utf-8")

messages= []

add_user_message(messages, [
    {
        "type": "image",
        "source":{
                "type": "base64",
                "media_type": "image/png",
                "data": image_bytes
            }
    },
    {
        "type": "text",
        "text": prompt
    }
])

chat(messages)





Message(id='msg_01Rbq7j91R5PHVwNg9UeeHJ5', content=[TextBlock(citations=None, text='## Property Analysis\n\n1. **Residence identification**: The primary residence appears to be a light-colored structure with multiple roof sections situated in the center of the image, connected to what looks like a driveway or cleared area.\n\n2. **Tree overhang analysis**: There are multiple trees surrounding the residence with canopies that appear to extend over portions of the roof, covering approximately 25-50% of the roof area, particularly on the northern and eastern sides.\n\n3. **Fire risk assessment**: The overhanging trees create potential ember catch points on the roof and form bridges between surrounding vegetation and the structure, though specific roof openings like chimneys cannot be clearly identified in this aerial view.\n\n4. **Defensible space identification**: The property shows a concerning pattern of connected tree canopies surrounding the home with relatively dense vegetation cove

In [6]:
with open("earth.pdf", "rb") as f:
    file_bytes = base64.standard_b64encode(f.read()).decode("utf-8")

messages= []

add_user_message(messages, [
    {
        "type": "document",
        "source":{
                "type": "base64",
                "media_type": "application/pdf",
                "data": file_bytes
            },
            "title": "earth.pdf",
            "citations": {"enabled": True}
    },
    {
        "type": "text",
        "text": "How were Earth's atmosphere and oceans formed?"
    }
])

chat(messages)


Message(id='msg_01Wm3SEgDfEpbidhLTAE9fRJ', content=[TextBlock(citations=None, text="Earth's atmosphere and oceans were formed through a complex process that occurred after the planet's initial formation.\n\n", type='text'), TextBlock(citations=[CitationPageLocation(cited_text="[42]\r\nEarth's atmosphere and oceans were formed by volcanic activity and outgassing.\r\n", document_index=0, document_title='earth.pdf', end_page_number=5, start_page_number=4, type='page_location')], text="Earth's atmosphere and oceans were formed by volcanic activity and outgassing.", type='text'), TextBlock(citations=None, text=' ', type='text'), TextBlock(citations=[CitationPageLocation(cited_text='[43] Water vapor from\r\nthese sources condensed into the oceans, augmented by water and ice from asteroids, protoplanets,\r\nand comets.\r\n', document_index=0, document_title='earth.pdf', end_page_number=5, start_page_number=4, type='page_location')], text='Water vapor from these sources condensed into the ocea